### Agents

#### Simple agent

In [ ]:
# Old-style agent initializer and agent types
from langchain.agents import initialize_agent, AgentType

# Decorator to convert a normal Python function into a LangChain tool
from langchain.tools import tool


# Load environment variables (.env)
load_dotenv()


# Create the Gemini language model instance
# model: which Gemini model to use
# temperature=0 makes responses deterministic and precise
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0
)


# Define a calculator tool
# @tool tells LangChain this function can be used by the agent
@tool
def calculator(expression: str) -> str: #this function takes a math expression as input and returns the result as a string
    """
    Useful for math calculations.
    Input: a math expression as a string
    Output: result as a string
    """
    try:
        # eval evaluates the math expression
        return str(eval(expression))
    except:
        # If the expression is invalid, return an error message
        return "Invalid expression"


# Define a tool to calculate word length
@tool
def get_word_length(word: str) -> str:
    """
    Returns the length of a word.
    Input: a string
    Output: length of the string
    """
    return str(len(word.strip()))


# List of tools that the agent is allowed to use
tools = [calculator, get_word_length]


# Initialize an agent
# tools: what functions the agent can call
# llm: the language model driving reasoning
# agent type: ZERO_SHOT_REACT_DESCRIPTION means
#   - the agent reasons step by step
#   - decides which tool to call based on description
# verbose=True prints the agent's internal reasoning
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


# Run the agent with a natural language query
# The agent will:
# 1. Understand the question
# 2. Decide which tool to use
# 3. Call the tool
# 4. Combine results into a final answer
result = agent.run(
    "What is 25 * 4, and how many letters are in the word 'LangChain'?"
)


# Print the final response produced by the agent
print("\nFinal Answer:", result)




> Entering new AgentExecutor chain...
Action: calculator
Action Input: 25 * 4
Observation: 100
Thought:I have calculated the first part of the question. Now I need to find the number of letters in the word 'LangChain'.
Action: get_word_length
Action Input: LangChain
Observation: 9
Thought:I have calculated both parts of the question.
Final Answer: 25 * 4 is 100, and there are 9 letters in the word 'LangChain'.

> Finished chain.

Final Answer: 25 * 4 is 100, and there are 9 letters in the word 'LangChain'.


#### Modern way: requires new version of langchain

In [ ]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import tool

# Load environment variables
load_dotenv()

# Initialize Gemini LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0
)

# Calculator tool
@tool
def calculator(expression: str) -> str:
    """Useful for math calculations."""
    try:
        return str(eval(expression))
    except Exception:
        return "Invalid expression"

# Word length tool
@tool
def get_word_length(word: str) -> str:
    """Returns the length of a word."""
    return str(len(word.strip()))

# Tools list
tools = [calculator, get_word_length]

# Initialize agent (legacy but supported)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Run the agent
result = agent.run(
    "What is 25 * 4,if answer is greater than 100 answer:who is president of INDIA else Answer:who is president of USA.?"
)

print("\nFinal Answer:", result)


In [ ]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import tool
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

# LLM
gemini = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0
)

@tool
def calculator(input: str) -> str:
    try:
        return str(eval(input))
    except Exception:
        return "Invalid Expression"

@tool
def word_length(input: str) -> str:
    return str(len(input.strip()))

tools = [calculator, word_length]

template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... 
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate.from_template(template)

agent = initialize_agent(
    tools=tools,
    llm=gemini,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    agent_kwargs={"prompt": prompt},
    verbose=True
)

query = "what is 9*99 if answer is <1000 answer: who is president of India else answer: who is president of US"
result = agent.invoke({"input": query})

print(result)
